In [301]:
# response.status_code
# response.content

In [65]:
import json
import requests
import pandas as pd
import psycopg2
from pandas import json_normalize
from sqlalchemy import create_engine
    
def prcImportFromMoex(dateStart,dateEnd):
    global i, imax, dfful
    print ('Start procedure prcImportFromMoex on i = '+str(i)+' - dates: '+ dateStart+' - '+dateEnd+' ...')

    
    url = 'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/WHFOB.jsonp?from='+dateStart+'&till='+dateEnd+'&limit=10000&sort_order=TRADEDATE&sort_order_desc=desc&iss.meta=off&iss.json=extended'
    #print ('url = '+ url)

    fname="dop_files/wheat_msk.json"
    response = requests.get(url)

    #запись в файл
    with open(fname, "wb") as f:
        f.write(response.content)
    f.close()

    #чтение из файл и обработка переноса строки и ещё первой строки
    text=''
    with open(fname, "r", encoding='utf-8') as f:
        for line in f:
            line=line.strip()
            line=line.replace('{"charsetinfo": {"name": "utf-8"}},','')
            text=text+line.strip()

    text=text[1:len(text)-1]
   
    text = json.loads(text)
    df = json_normalize(text,'history')    
                
    if i==1:
        print('Init dfful...')
        dfful=df  
    else: 
        print('Adding df to dfful...')
        dfful=pd.concat([dfful, df])

    if i==imax:
        print ('Creating csv file for dataframe...')
        dfful.to_csv('../project_otus_dbt/data/source_wheat_msk.csv',index=False)
        
    print ('End procedure prcImportFromMoex success')
    i=i+1
    

In [66]:
#init
i=1
imax=1

#exec procedure 
prcImportFromMoex(dateStart='2021-12-02',dateEnd='2022-02-07') 


print ('End all success')

Start procedure prcImportFromMoex on i = 1 - dates: 2021-03-22 - 2021-06-01 ...
Init dfful...
End procedure prcImportFromMoex success
Start procedure prcImportFromMoex on i = 2 - dates: 2021-06-02 - 2021-09-01 ...
Adding df to dfful...
End procedure prcImportFromMoex success
Start procedure prcImportFromMoex on i = 3 - dates: 2021-09-02 - 2021-12-01 ...
Adding df to dfful...
End procedure prcImportFromMoex success
Start procedure prcImportFromMoex on i = 4 - dates: 2021-12-02 - 2022-02-07 ...
Adding df to dfful...
Creating csv file for dataframe...
End procedure prcImportFromMoex success
End all success
